
# 📓 阶段 1 · 1.3 专用 Notebook —— 混杂与偏倚 · 迷你实验合集

本 Notebook 包含：  
1) **混杂实验**：展示“重症者更常被治疗 → 天真比较方向颠倒”并用回归/IPTW矫正；  
2) **碰撞实验**：展示“条件化碰撞变量 → 凭空造出相关”；  
3) **变量选择清单**：避免过度/不足/错误调整的操作指南。

> 运行顺序：从 1 到 3 依次执行。



## 1. 混杂实验：方向颠倒与校正
**设定**：重症度 `Z` 越高 → 越可能被治疗 `X`，也越可能死亡 `Y`；真相中治疗**降低**死亡。  
**现象**：天真比较显示“治疗更致死”（被混杂骗了）；回归/IPTW 校正后恢复“治疗保护”。


In [ ]:

import numpy as np, pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

np.random.seed(0)
n = 5000

# 混杂 Z：重症度（越高越重）
Z = np.clip(np.random.normal(0, 1, n), -2.5, 2.5)

# 治疗分配：Z 越高越可能被治疗（指征混杂）
logit_t = -0.2 + 1.2*Z
p_t = 1/(1+np.exp(-logit_t))
X = (np.random.rand(n) < p_t).astype(int)  # 1=治疗

# 真相：治疗有益（降低死亡），但重症本身提高死亡
logit_y = -2.0 + 1.1*Z - 0.8*X
p_y = 1/(1+np.exp(-logit_y))
Y = (np.random.rand(n) < p_y).astype(int)  # 1=死亡

df = pd.DataFrame({'Z':Z,'X':X,'Y':Y})

# 1) 天真比较：治疗组 vs 对照组死亡率差
naive_diff = df.loc[df.X==1,'Y'].mean() - df.loc[df.X==0,'Y'].mean()
print("天真比较：治疗组-对照组死亡率差 =", round(naive_diff, 3))

# 2) 回归调整：在 logit(Y) ~ X + Z 中看 X 的系数方向
m_adj = LogisticRegression(max_iter=1000).fit(df[['X','Z']], df['Y'])
print("Logit回归中 X 的系数（<0 表示治疗保护）:", round(m_adj.coef_[0][0], 3))

# 3) IPTW：只用 Z 建 PS，并估计 ATE（死亡率差）
scaler = StandardScaler()
ps = LogisticRegression(max_iter=1000).fit(scaler.fit_transform(df[['Z']]), df['X']).predict_proba(scaler.transform(df[['Z']]))[:,1]
w = np.where(df['X']==1, 1/ps, 1/(1-ps))
ate_iptw = (w[df.X==1]*df.Y[df.X==1]).sum()/w[df.X==1].sum() - (w[df.X==0]*df.Y[df.X==0]).sum()/w[df.X==0].sum()
print("IPTW 估计ATE（>0=增风险，<0=降风险）:", round(ate_iptw, 3))



## 2. 碰撞实验：条件化共同结果 → 造出虚假相关
**设定**：X 与 U 原本独立；A 是 X 与 U 的共同结果（**碰撞**）。若只分析 A=1 的样本，就会凭空出现 X–Y 的相关。


In [ ]:

import numpy as np, pandas as pd

np.random.seed(1); n=100000

# X 与 U 互不相关
X = np.random.binomial(1, 0.5, n)
U = np.random.binomial(1, 0.5, n)

# A 是 X 与 U 的共同结果（碰撞）：只有 A=1 的人进入样本
A = (np.random.rand(n) < (0.05 + 0.4*X + 0.4*U)).astype(int)
keep = A==1

# 结局只受 U 影响（与 X 无关），便于观察碰撞偏倚
Y = U  # 简化设定

dfc = pd.DataFrame({'X':X[keep], 'U':U[keep], 'Y':Y[keep]})
overall_corr = np.corrcoef(X, Y)[0,1]
sample_corr = np.corrcoef(dfc.X, dfc.Y)[0,1]
print("总体 X–Y 相关（应≈0）→", round(overall_corr, 4))
print("筛选 A=1 后样本的 X–Y 相关（被碰撞打开）→", round(sample_corr, 4))



## 3. 变量选择清单（避免过度/不足/错误调整）

**只把“治疗前”变量当混杂**；优先选择**同时影响 X 与 Y 的共同原因**；避免把**中介**和**碰撞**纳入 PS/背门调整集。

### 3.1 判定角色
- 混杂（Confounder）：治疗前、共同原因（✅ 调整）  
- 中介（Mediator）：X→M→Y（❌ 若估总效应，不在 PS/背门里调整）  
- 碰撞（Collider）：X→A←U 的共同结果（❌ 不能条件化）  
- 效应修饰（Effect Modifier）：效应随 W 改变（⚠️ 报异质性/CATE）

### 3.2 最小流程
1) 明确时间线：变量是否在治疗前可观测；  
2) 画简易 DAG：列共同原因，排除中介与碰撞；  
3) 仅用治疗前混杂建 PS，检查**平衡（SMD）**与**重叠（PS 分布）**；  
4) 估计效应：PSM/IPTW/DR/G-formula；  
5) 敏感性分析：极端权重截尾、替代模型、未测混杂/负控。

> 建议把本页当作“作战手册”，每做一次分析都过一遍这张清单。
